In [1]:
#Imports
import pandas as pd
import os.path
import random
import time
import string
import webbrowser
import numpy as np
import matplotlib.pyplot as plt
import string

#All Function definitions

def generateIDs(keyname, numberOfQuestions):
    '''generates a unique ID for each question using the beginning characters of each question'''
    goodIDs=[]
    with open(keyname) as file:  
        data = file.read()
    for n in range(1,numberOfQuestions+1):
            #print("processing: "+str(n))
            locs=[]
            occursat=find_all(data, str(n)+')')
            for ii in occursat:
                if data[ii-1]=='\r' or data[ii-1]=='\n'  and data[ii+3]!="_":# check for both /r and /n
                    locs.append(ii)
                    #print(locs)
            z=lambda x: str('0')+str(x) if x<10 else str(x)
            theID=z(n)+"_"+(data[locs[0]+3:locs[0]+50]).strip()
            #print(theID)
            goodIDs.append(theID)
    return goodIDs

def unduplicate(dup_list):
    '''finds duplicate entries in a list and adds characters to duplicated entries to make them unique.
    works in a roundabout way by converting to a dataframe first , and then converting back to a list in the end'''
    df = pd.DataFrame(dup_list)
    dups=df[df.duplicated(keep='first')]
    if not dups.empty:
        dup_indices=list(dups.index)
        for i in dup_indices:
            df.iat[i,0]=df.iat[i,0]+str(i)
            print("Unduplicating: "+ df.iat[i,0])
    undup_list=list(df.values.flatten())
    return undup_list

def createQuestionIDs(numberOfQuestions):
    '''this will generateIDs unique question IDs for each question from the raw keys'''
    allIDs=[]
    for n in range(0,6):
        filename="key"+str(n)+".txt"
        if (os.path.isfile(filename)): 
            qid=generateIDs(filename,numberOfQuestions)
            allIDs.append(qid)
    allIDs[0]=unduplicate(allIDs[0])
 #The ID's generated are slightly different in each version, so set them all equal to the ID's in version0           
    for n in range(1,numberOfVersions):
        for m in range(0,numberOfQuestions):
            sub=allIDs[0][m][5:len(allIDs[0][m])-len(allIDs[0][m])//5]
            ########### THe 5 and 10 are chosen without any logic - some logic would be nice
            #print(sub)
            matching = [s for s in allIDs[n] if sub in s]
            found_in=allIDs[n].index(matching[0])
            if allIDs[n][found_in]!=allIDs[0][m]:
                    #print(allIDs[n][found_in]+"   changed to    "+allIDs[0][m])
                    allIDs[n][found_in]=allIDs[0][m]
                    
    
    return allIDs

def find_all(a_str, sub):
    '''finds all instances of a substring in a string'''
    start = 0
    result=[]
    while True:
        start = a_str.find(sub, start)
        if start == -1: return result
        result.append(start)
        start += len(sub) # use start += 1 to find overlapping matches
    return result

def cleanKey(key_name):
    '''retrieves just the "key" part of exported test. 
    Just reads and returns the key if the questions have already been deleted 
    writes to a new file with name cleankey'''
    if(os.path.isfile(key_name)):
        #print("Scrubbing: "+key_name+" which is :" + str(type(key_name)))
        with open(key_name) as file:  
            data = file.read()
            p=data.rfind('_____')
            if p!=-1: 
                data=data[p+5:]#delete all the junk before "_______"
                data=data[data.find('1)'):]#keep all the stuff after the 1)
        endline=['\rPoints: ', '\nPoints: ']
        for pp in endline:
            if data.find(pp)!=-1:# remove the question ID to a separate column if necessary
                data = data.replace(pp,'\t')
        with open('cleaned_'+key_name, 'w') as file:
            file.write(data)
    else:
        print(str(key_name)+" not found")
        
def getAllKeys(fixed_points_per_question=1):
    '''read all files named key0-key4, make them into strings and and return a dictionary
    containing a list of keys and also the number of questions'''
    keylist=[]
    pointlist=[]
    numberOfQuestions=0

    for n in range(0,6):
        filename="key"+str(n)+".txt"
        #print("processing: "+filename)
        if os.path.isfile(filename): 
            cleanKey(filename)#always create a fresh cleankey, to avoid using an outdated one
            pp=pd.read_table("cleaned_"+filename, header=None)
            keys=pp.iloc[:,0].to_frame()#converts a series to a dataframe
            keylist.append(makekey(keys))
            if pp.shape[1]>1:
                points=pp.iloc[:,1].to_frame()
                pointlist.append(makepoints(points))
            else:
                points=[fixed_points_per_question]*pp.shape[0]
                points[-1]=0
                pointlist.append(points)

    numberOfQuestions=keys.shape[0]
    if not keylist:
        print("Answerkeys not found. Make sure they are named key0.txt, key1.txt etc")
    getAllKeysOutput={"keylist":keylist,"pointlist":pointlist,"numberOfQuestions":numberOfQuestions}
    return getAllKeysOutput


def makekey(key_from_test):
    '''key_from_test is a dataframe formed from reading the answerkey file. This function converts it into a text string with numbers 0-5 representing A-E'''
    #keydictionary={"A":"0","B":"1","C":"2","D":"3","E":"4"}
    thekey=""
    for i in range(0,key_from_test.shape[0]):
        #print("At i="+str(i))
        #get the last letter, i.e. the "C" from "1)C" 
        thekey+=keydictionary[key_from_test.iat[i,0][-1:]]
    return thekey

def makepoints(points):
    '''pointlist_from_test is a dataframe formed from reading the answerkey file. This function converts it into a text string with numbers 0-5 representing A-E'''
    #keydictionary={"A":"0","B":"1","C":"2","D":"3","E":"4"}
    pointlist=[]
    for i in range(0,points.shape[0]):
        #print("At i="+str(i))
        #get the last letter, i.e. the "C" from "1)C" 
        pointlist.append(int(points.iat[i,0]))
    pointlist[-1]=0
    return pointlist


def process_grades(data,outs, QIDs,analysis=False):
    '''grades all exams using correct keys, writes questions missed and scores'''

    for NN in range(0, data.shape[0]):
        #print("NN: "+str(NN))
        ans=data.iat[NN,2]
        if len(ans)==numberOfQuestions-1:
            v=guess_the_version(ans)
            ans=ans+str(v)
            print("Assuming version "+invkeydictionary[v]+ " for: "+data.iat[NN,1]+" (Srl No: "+data.iat[NN,0]+").")
        check1=gradeWithKeylist(ans, outs, QIDs, analysis, NN)
        data.iat[NN,3]=check1['missed']
        data.iat[NN,4]=check1['score']
        data.iat[NN,5]=100.0*float(check1['score'])/float(new_totalpoints)

    if analysis:
        #remove the column with the version numbers
        analysis_df.drop(analysis_df.columns[analysis_df.shape[1]-2], axis=1, inplace=True)
    return data


def gradeWithKeylist(ans,outs, QIDs, analysis=False, N=0):
    '''multiple versions - find the correct key as indicated on the last question on the exam '''
    keylist=outs["keylist"]
    pointlist=outs["pointlist"]
    numberOfQuestions=outs["numberOfQuestions"] 
    
    assert(keylist!=[])
    assert (ans[-1:] in ['0','1','2','3','4'])
    whichKey=int(ans[-1:])
    
    key=keylist[whichKey]
    points=pointlist[whichKey]
    assert len(key)==len(ans)
    missed=""
    rejalt=[1]*numberOfQuestions
    for n in range(0,len(key)-1):
        if key[n]!=ans[n]:
            #print("storing")
            missed+=str(n+1)+", "
            rejalt[n]=0
    if sum(rejalt)==numberOfQuestions:
        missed="ALL CORRECT"
    else:
        missed="v"+invkeydictionary[whichKey]+": "+missed[:len(missed)-2]
    score=sum([i*j for i,j in zip(points,rejalt)])
    

    mydict1 = dict(zip(QIDs[whichKey],rejalt))
    sortedIDs=sorted(mydict1.keys())
    sorted_rejalt=[mydict1[k] for k in sortedIDs]####its sorted according to v0
    

    
    if analysis:   
        sorted_rejalt.append(score)
        analysis_df.loc[N] = sorted_rejalt 
        
        mydict2 = dict(zip(QIDs[whichKey],ans))  
        sortedIDs=sorted(mydict2.keys())
        sorted_ans=[mydict2[k] for k in sortedIDs]
        allAnswers_df.loc[N] = sorted_ans
    

        
    return {'missed':missed, 'score':score}


def guess_the_version(no_version):
    scores=[]
    for n in range(numberOfVersions):
        try_version=no_version+str(n)
        check1=gradeWithKeylist(try_version, outs, QIDs, analysis=False)
        scores.append(check1['score'])
    return scores.index(max(scores))

def count_how_many(col, value):
    '''takes a pandas series "col" and counts the number of instances of a "value" 
    Not used since pandas has a built-in function'''
    
    count=0
    for n in range(0,col.shape[0]):
        if col[n]==value:
            count+=1
    return count


def addStarsToCorrectChoices(rdf, keylist, m, QIDS):
    '''adds stars to correct choices in rdf'''
    for n in range(0,rdf.shape[0]):
        if m!=0:
            mydict0 = dict(zip(QIDs[m],keylist[m])) 
            sortedkey=[mydict0[x] for x in QIDs[0]]
        else:
            sortedkey=keylist[m]
        the_correct_answer=sortedkey[n]
        rdf.iloc[n][the_correct_answer]=rdf.iloc[n][the_correct_answer]+"*"
    return rdf

def analyse_items(a_df, QIDs):
    '''create separate dfs with how many marked correct for each version separately'''
    vers=['0','1','2','3','4']
    lvers=['A','B','C','D','E']
    outvars=[]
    print("Versions found:")
    for n in range(0,len(vers)):
        ch=str(n)
        ##check the last column "which version..." to find the version and see if it matches ch
        #part_df = a_df.loc[allAnswers_df[list(allAnswers_df)[-1]]==ch]
        part_df = a_df.loc[a_df[list(a_df)[-1]]==ch]
        if not part_df.empty:
            print(lvers[n])
            part_df=part_df.T
            part_df=part_df.apply(pd.Series.value_counts, axis=1).fillna(0)
            part_df=part_df.applymap(int)
            part_df=part_df.applymap(str)
            part_df=addStarsToCorrectChoices(part_df,keylist,n, QIDs)
            if ' ' in list(part_df):
                part_df.rename(columns={' ': 'blank'}, inplace=True)   
            for pp in vers:
                if pp in list(part_df):
                    part_df.rename(columns={pp: lvers[int(pp)]}, inplace=True)
            outvars.append(part_df)
        
    return outvars

def make_item_analysis(a_df):
    
    item_analysis_df=pd.DataFrame(index=["Difficulty","Discrimination"],columns = list(analysis_df))
    n=a_df.shape[0]
    v=analysis_df.values#converts into np array
    
    
    #Calculating Difficulty
    u1=np.sum(v,axis=0)/n
    item_analysis_df.loc["Difficulty"]=u1[0:u1.shape[0]]
    
    #Calculating Discrimination
    students_per_group=n//3
    if students_per_group<2:
        print("Need more students for discrimination analysis (at least 7).")
    else:
        v=v[0:v.shape[0]-1,:]#exclude the bottom row which had the sums (total number correct for each q)
        v=v[v[:,v.shape[1]-1].argsort()[::-1]]#sort descending by scores
        #print(v)
        u1=np.sum(v[0:students_per_group,:],axis=0)#top scorers
        u2=np.sum(v[n-students_per_group : n,:],axis=0)#bottom scorers
        disc=(u1-u2)/students_per_group
        disc.shape
        item_analysis_df.loc["Discrimination"]=disc[0:disc.shape[0]]
    
    item_analysis_df.drop("score", axis=1, inplace=True)
    return item_analysis_df.T

#################Writing data#############################
def write_to_xl(adf, rfilename):
    adf.rename(columns={'Srl No': 'Serial Number Text Grade <Text>'})#This facilitates the Vlookup later
    writer = pd.ExcelWriter(rfilename+'_processed.xlsx')
    adf.to_excel(writer,'Sheet1',index=False)
    writer.save()
    
def write_to_csv(adf, rfilename):
    filename = rfilename+'_processed.csv'
    #adf=adf.rename(columns={'Srl No': 'Serial Number Text Grade <Text>'})#This facilitates the Vlookup later
    adf.to_csv(filename, index=False)
    

def write_to_webpage(j_df, a_df,ia_df,ncbv, histobin=11):
    filename = rawdatafilename+'_summary.html'
    f = open(filename,'w')
    
    uu=df['Percentage'].describe().to_frame()
    uu.rename(columns={'Percentage': "Value"}, index={'count':'Number of Students','mean':'Mean(%)', 'std':'Standard Deviation', 'min':'Lowest(%)','25%':'25th percentile', '50%':'50th percentile', '75%':'75th percentile', 'max':"Highest(%)"},inplace=True)
    uu.loc['Maximum Available(%) '] = [100*totalpoints/new_totalpoints]
    uu.loc['Points dropped '] = [point_drop]

    pre="<h2>Summary Data:</h2>"
    summary_table=uu.to_html(float_format=lambda x: '%10.2f' % x).replace("dataframe" ,"sumdat")
    summary_table=pre+summary_table
    post="<p>Score Distribution:</p>"
    
    figure = plt.figure()
    df['Percentage'].plot(kind='hist', bins=10)
    plt.xlim(xmax=100*totalpoints/new_totalpoints)
    figure.savefig('histo.svg')
    
    pre="<h2>Score Distribution:</h2>"
    image_code='<img src="histo.svg" alt="histogram">'
    image=pre+image_code

    pre="<h2>Item Analysis:</h2>"
    item_analysis_table=ia_df.sort_values('Difficulty').to_html(float_format=lambda x: '%10.2f' % x).replace("dataframe" ,"customers")
    item_analysis_table=pre+item_analysis_table

    missed_tables=""
    for nc in range(len(ncbv)):
        pre="<h2>Distractor Analysis for Version "+invkeydictionary[nc]+":</h2>"
        post=ncbv[nc].to_html().replace("dataframe" ,"howmany")
        missed_tables+=pre+post


    html_start = """<html>
    <head>
     <link rel="stylesheet" type="text/css" href="mystyle.css">
    </head>
    <body>
    <h1>Data Analysis</h1>
    """


    html_end='''</body>
    </html>'''
    message=html_start+summary_table+image+item_analysis_table+missed_tables+html_end
    f.write(message)
    f.close()

    #Change path to reflect file location
    webbrowser.open_new_tab(filename)

###############################################
## Functions that create fake data#############

def create_afake(number_of_versions, blanks=False):
    '''creates a fake answerkey'''
    ansstring=[]
    if not blanks:
        choices=[0,1,2,3,4]
    else:
        choices=[0,1,2,3,4,' ']
    for n in range(25):
        ans=random.randrange(0,len(choices))
        ansstring.append(str(choices[ans]))
    ans=random.randrange(0,number_of_versions)
    ansstring.append(str(ans))
    ansstring=''.join(ansstring)
    return ansstring
def create_fakes(N, number_of_versions, blanks=False):
    '''creates fakes and writes to an excel file'''
    fakes_df=pd.DataFrame(index=range(N),columns = range(3),dtype=str)
    fakes_df.iloc[:,0:2] =""
    for i in range(0,N):
        fakes_df.iat[i,2]=create_afake(number_of_versions, blanks)
    writer = pd.ExcelWriter('fakes.xlsx')
    fakes_df.to_excel(writer,'Sheet1')
    writer.save()

########################################################
############### Checking for incorrect Srl Nos################

def find_oks(row):
    return row['Name'].find(row['First Name'].upper())!=-1
def remove_ok_rows(df):
    notnull=df.dropna()
    okrows=notnull.loc[notnull.apply(find_oks, axis=1)]
    oksremoved=pd.concat([df, okrows]).drop_duplicates(keep=False)
    return oksremoved

def check_serial_numbers():
    df=pd.read_csv('BSGrades.csv', usecols=["Last Name", "First Name", "Serial Number Text Grade <Text>"])
    df_all=pd.read_excel('All.xlsx', header=None, parse_cols=1,names = ["Serial Number Text Grade <Text>", "Name"])

    checker_left = pd.merge(df,df_all[['Serial Number Text Grade <Text>','Name']],on='Serial Number Text Grade <Text>', how='left')
    checker_right = pd.merge(df,df_all[['Serial Number Text Grade <Text>','Name']],on='Serial Number Text Grade <Text>', how='right')
    u1=checker_left[checker_left.isnull().any(axis=1)]
    u2=checker_left[checker_left['Serial Number Text Grade <Text>'].duplicated(keep=False)]
    u3=checker_right[checker_right.isnull().any(axis=1)]
    error_rpt=pd.concat([u1, u2, u3], axis=0)
    if not error_rpt.empty:
        error_rpt=remove_ok_rows(error_rpt)
    error_rpt["Last Name"] = error_rpt["Last Name"]+' '+error_rpt['First Name']
    error_rpt.drop('First Name', axis=1,inplace=True)
    error_rpt.rename(columns={'Serial Number Text Grade <Text>': 'Srl No', 'Last Name':'Registered Name', 'Name':'Entered Name'}, inplace=True) 
    return error_rpt

Manually convert the dat file into an excel file using excel. Only extract the serial number,
name and Answers, and make sure Answers is text.

In [2]:
#COLLECTING DATA
rawdatafilename='All'
xls_file = pd.ExcelFile(rawdatafilename+'.xlsx')
df = xls_file.parse('Sheet1', header=None, parse_cols=2,names = ["Srl No", "Name", "Answers"], dtype='str')
#parse_cols makes sure that only cols 0,1 and 2 are extracted
#checking for blanks, print only if blanks found
if not df[df['Answers'].str.contains(" ")].empty:
    blankers=df[df['Answers'].str.contains(" ")]
    display(blankers)
ER=check_serial_numbers()
if not ER.empty:
    display(ER)
df["Missed"] = ""
df["Score"]=0
df["Percentage"]=np.nan

,Srl No,Name,Answers
2,1009,HAN KARIN,0110241 21 4040


,Registered Name,Srl No,Entered Name
1,Bolling Atiyaa,1003,NaN
12,Muezzinoglu Mine,1018,NaN
3,Carlson Brynn,1002,MUEZZINOGLU MINE


In [3]:
#initializing everything
keydictionary={"A":"0","B":"1","C":"2","D":"3","E":"4"}
invkeydictionary={0:"A",1:"B",2:"C",3:"D",4:"E"}
points_per_question=3
point_drop=2
outs = getAllKeys(points_per_question)
keylist=outs["keylist"]
pointlist=outs["pointlist"]
numberOfQuestions=outs["numberOfQuestions"]
if not keylist or not pointlist or numberOfQuestions==0:
    sys.exit("Keys not properly imported")
totalpoints=sum(pointlist[2])
new_totalpoints=totalpoints-point_drop
numberOfVersions=len(keylist)
numberOfStudents=df.shape[0]
QIDs=createQuestionIDs(numberOfQuestions)
headings=list(QIDs[0])#####headings=QIDs[0] assigns by reference, so changing headings will change QIDs[0]
allAnswers_df = pd.DataFrame(index=range(numberOfStudents), columns = headings)# stores all student answers for each question, questions are the columns
#results_df = pd.DataFrame(index=headings, columns = ["A","B","C","D","E","Skipped","Diff","Disc"])
headings.append("score")
analysis_df  = pd.DataFrame(index=range(numberOfStudents),columns = headings)# stores whether answer was correct, questions are the columns, last column is the score


In [4]:
##Everything initialized. Running the code now
didnt_enter_versions=df[df['Answers'].map(len)!=numberOfQuestions]
if not didnt_enter_versions.empty:
    didnt_enter_versions=didnt_enter_versions[['Srl No', 'Name','Answers']]
    display(didnt_enter_versions)
print("Starting calculations...")
starttime = time.time()
df=process_grades(df,outs, QIDs, analysis=True)
number_correct_by_version=analyse_items(allAnswers_df, QIDs)
item_analysis_df = make_item_analysis(analysis_df) 
endtime = time.time()
print("Done. That took: "+str(endtime-starttime)+ " sec for "+str(numberOfStudents)+" students")
#check for duplicated serial numbers
if not df[df['Srl No'].duplicated(keep=False)].empty:
    print('Warning: Duplicates in serial numbers found!')
    dupes=df[df['Srl No'].duplicated(keep=False)]
max_score=analysis_df.at[analysis_df['score'].idxmax(),'score']

,Srl No,Name,Answers
5,1002,MUEZZINOGLU MINE,02022401103400
18,1001,ACQUAYE AMBER,23100110212344


Starting calculations...
Assuming version B for: MUEZZINOGLU MINE (Srl No: 1002).
Assuming version A for: ACQUAYE AMBER (Srl No: 1001).
Versions found:
A
B
C
D
Done. That took: 0.1047203540802002 sec for 19 students


In [5]:
write_to_csv(df,rawdatafilename)
write_to_webpage(df, analysis_df, item_analysis_df, number_correct_by_version)

In [ ]:
#Grades and displays the N'th entry in the list using a keylist
N=random.randint(1,df.shape[0]-1)
#for N in range(df.shape[0]-1):
check1=gradeWithKeylist(df.iat[N,2], outs, QIDs, analysis=False)
print("Row "+ str(N)+": "+df.iat[N,1]+", "+str(df.iat[N,0])+". Missed "+str(check1['missed'])+ ". Scored "+str(check1['score'])+"/"+str(totalpoints))
print('That is: '+str(100*check1['score']/new_totalpoints)+'% with '+str(point_drop)+' points dropped')

In [ ]:
didnt_enter_versions=didnt_enter_versions[['Srl No', 'Name','Answers']]

In [ ]:
didnt_enter_versions